# Creazione Dataset

In [ ]:
import os
from google.colab import drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
from pandas import read_csv, DataFrame, concat
from IPython.display import display, clear_output, HTML
from ipywidgets import Label, RadioButtons, Button, Output, VBox, HBox, Layout

# Definizione della funzione manual_labeling per l'etichettatura manuale dei file
def manual_labeling(folder_path, categories, subcategories, saved_data_path):

    # Caricamento dei dati etichettati esistenti se esistono, altrimenti inizializzazione di un set vuoto per i file esistenti

    if os.path.exists(saved_data_path):
        existing_data = read_csv(saved_data_path)
        existing_files = set(existing_data['File'])
    else:
        existing_files = set()

    data = {'File': [], 'Category': [], 'Subcategory': [], 'Content': []}

    # Creazione dei widget per l'etichettatura manuale
    file_label = Label(value='', layout=Layout(margin='0 0 10px 0'))
    category_radio = RadioButtons(options=categories, description='Categoria:', layout=Layout(margin='0 0 10px 0'))
    subcategory_radio = RadioButtons(options=[], description='Sottocategoria:', layout=Layout(margin='0 0 10px 0'))
    content_output = Output(layout=Layout(margin='10px 0 10px 0'))
    copy_to_clipboard_button = Button(description='Copia negli Appunti', disabled=True, layout=Layout(margin='10px 10px 10px 0'))
    confirm_button = Button(description='Conferma', disabled=True, layout=Layout(margin='10px 10px 10px 0'))
    save_button = Button(description='Salva', disabled=True, layout=Layout(margin='10px 10px 10px 0'))

    # Widget di output per visualizzare messaggi
    message_output = Output()

    # Widget del contatore
    counter_label = Label(value='File nel Database: 0, File da Etichettare: 0', layout=Layout(margin='0 0 10px 0'))

    # Funzione per aggiornare il nome del file e le sottocategorie
    def update_widgets(file_name):
        global content
        file_label.value = f'File: {file_name}'
        subcategory_options = subcategories.get(category_radio.value, [])

        # Impostazione dinamica del layout in base alla lunghezza delle opzioni delle sottocategorie

        subcategory_radio.layout.flex_flow = 'column'  # Visualizzazione verticale

        subcategory_radio.options = subcategory_options

        # Aggiornamento del contatore
        update_counter()

        # Lettura e visualizzazione del contenuto del file
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()

        # Visualizzazione del contenuto in una riga separata
        with content_output:
            clear_output(wait=True)
            print(f'Contenuto:\n{content}')


        copy_to_clipboard_button.disabled = False

    # Funzione per aggiornare l'etichetta del contatore
    def update_counter():
        files_in_db = len(existing_files)
        files_to_label = len([file for file in os.listdir(folder_path) if file not in existing_files])
        counter_label.value = f'File nel Database: {files_in_db}, File da Etichettare: {files_to_label}'

    # Funzione per aggiornare lo stato dei pulsanti "Conferma" e "Salva"
    def update_confirm_save_button_state():
        global content
        confirm_button.disabled = not content  # Disabilita il pulsante "Conferma" se il contenuto non è disponibile
        save_button.disabled = not content  # Disabilita il pulsante "Salva" se il contenuto non è disponibile

    # Gestore degli eventi per il cambio di categoria
    def category_change(change):
        subcategory_radio.options = subcategories.get(change.new, [])
        subcategory_radio.value = None
        update_copy_button_state()
        update_confirm_save_button_state()

    # Funzione per aggiornare lo stato dei pulsanti
    def update_copy_button_state():
        global content
        if content:  # Verifica se il contenuto non è vuoto
            copy_to_clipboard_button.disabled = False  # Abilita il pulsante se il contenuto è disponibile
        else:
            copy_to_clipboard_button.disabled = True  # Disabilita il pulsante se il contenuto non è disponibile
    # Gestore degli eventi per il pulsante
    def copy_to_clipboard_button_click(_):
        global content
        with content_output:
            clear_output(wait=True)
            print(f'Contenuto:\n{content}')

        copy_text_to_clipboard(content)
        update_message('Contenuto copiato negli Appunti!')

        copy_to_clipboard_button.disabled = False

    # Gestore degli eventi per il pulsante "Conferma"
    def confirm_button_click(_):
        file_name = file_label.value.split(': ')[1]
        data['File'].append(file_name)
        data['Category'].append(category_radio.value)
        data['Subcategory'].append(subcategory_radio.value)

        # Leggi il contenuto del file
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            data['Content'].append(content)

        # Aggiorna l'insieme dei file esistenti
        existing_files.add(file_name)
        update_counter()

        try:
            update_widgets(next(file_iterator))
            confirm_button.disabled = not content  # Disabilita il pulsante "Conferma" quando si passa al file successivo
            save_button.disabled = not content  # Disabilita il pulsante "Salva" quando si passa al file successivo
            update_copy_button_state()  # Aggiorna lo stato del pulsante "Copia negli Appunti"
            update_message('')  # Cancella l'output dei messaggi

        except StopIteration:
            # Nessun altro file, visualizza un messaggio
            update_message('Tutti i file etichettati!')
            save_button.disabled = not content  # Abilita il pulsante "Salva" alla fine
            update_message('Fase di etichettatura del database completata')
            category_radio.disabled = True  # Disabilita i RadioButtons quando l'etichettatura è completa
            subcategory_radio.disabled = True

    # Gestore degli eventi per il pulsante "Salva"
    def save_button_click(_):
        labeled_df = DataFrame(data)
        if os.path.exists(saved_data_path):
            existing_data = read_csv(saved_data_path)
            labeled_df = concat([existing_data, labeled_df], ignore_index=True)
        labeled_df.to_csv(saved_data_path, index=False)
        message_output.append_stdout(f'Dataset etichettato salvato in {saved_data_path}.')
        save_button.disabled = True

        # Aggiorna l'insieme dei file esistenti
        existing_files.update(data['File'])
        update_counter()
        update_copy_button_state()

    # Funzione per copiare il testo (soluzione alternativa per Colab)
    def copy_text_to_clipboard(text):
        js_code = f'''
            var textarea = document.createElement("textarea");
            textarea.value = `{text}`;
            document.body.appendChild(textarea);
            textarea.select();
            document.execCommand("copy");
            document.body.removeChild(textarea);
        '''
        display(HTML(f'<script>{js_code}</script>'))

    # Funzione per aggiornare il messaggio
    def update_message(message):
        with message_output:
            clear_output(wait=True)
            print(message)

    # Collegamento dei gestori degli eventi
    confirm_button.on_click(confirm_button_click)
    category_radio.observe(category_change, names='value')
    save_button.on_click(save_button_click)
    copy_to_clipboard_button.on_click(copy_to_clipboard_button_click)

    # Impostazione dello stato iniziale
    content = ""  # Inizializzazione del contenuto
    file_iterator = iter(file for file in os.listdir(folder_path) if file not in existing_files)
    try:
        update_widgets(next(file_iterator))
        update_copy_button_state()
        update_confirm_save_button_state()  # Stato iniziale dei pulsanti "Conferma" e "Salva"
    except StopIteration:
        update_message('Tutti i file etichettati!')
        save_button.disabled = True  # Abilita il pulsante "Salva" alla fine
        update_message('Fase di etichettatura del database completata')
        category_radio.disabled = True  # Disabilita i RadioButtons quando l'etichettatura è completa
        subcategory_radio.disabled = True

    # Visualizzazione dei widget con layout migliorato
    display(VBox([counter_label, file_label, content_output, HBox([category_radio, subcategory_radio, confirm_button, save_button, copy_to_clipboard_button]), message_output]))

# Esempio di utilizzo
folder_path = '/content/drive/MyDrive/dilbert-data'
categories_list = [
    'Corporate mood',
    'Technology and engineering',
    'Recurring characters',
    'Social criticism',
    'Political satire',
    'General humor'
]

subcategories_dict = {
    'Corporate mood': [
        'Corporate hierarchy',
        'Office politics',
        'Absurd corporate projects',
        'Group dynamics'
    ],
    'Technology and engineering': [
        'Technological problems',
        'Technologic innovations',
        'Stereotypes about engineers'
    ],
    'Recurring characters': [
        'Dilbert and his work adventures',
        'The stupid boss',
        'The lazy employee',
        'The competitive colleague',
        'Dogbert and other animal characters'
    ],
    'Social criticism': [
        'Bureaucracy',
        'Technology in society',
        'Business management'
    ],
    'Political satire': [
        'Reflections on political issues in the corporate world'
    ],
    'General humor': [
        'Comical and paradoxical situations',
        'Humor based on puns and absurd situations'
    ]
}

saved_data_path = '/content/drive/MyDrive/dilbert-topic-extractor/labeled_dataset.csv'

manual_labeling(folder_path, categories_list, subcategories_dict, saved_data_path)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Preparazione dei dati

In [ ]:
import os
from google.colab import drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
from pandas import read_csv

# Monta Google Drive
drive.mount('/content/drive')

# Percorso del dataset
dataset_path = '/content/drive/MyDrive/dilbert-topic-extractor/labeled_dataset.csv'

# Carica il dataset
dataset = read_csv(dataset_path)

# Visualizza le prime righe del dataset
# print(dataset.head())

# Conta le categorie e le sottocategorie uniche nel dataset
categorie_uniche = dataset['Category'].unique()
sottocategorie_uniche = dataset['Subcategory'].unique()

print("Categorie uniche:", categorie_uniche)
print("Sottocategorie uniche:", sottocategorie_uniche)

# Verifica dei valori mancanti nella colonna "Category"
missing_categories = dataset['Category'].isnull().sum()
print("Numero di valori mancanti nella colonna 'Category':", missing_categories)

# Verifica dei valori mancanti nella colonna "Subcategory"
missing_subcategories = dataset['Subcategory'].isnull().sum()
print("Numero di valori mancanti nella colonna 'Subcategory':", missing_subcategories)

# Verifica dei valori mancanti nella colonna "Content"
missing_subcategories = dataset['Content'].isnull().sum()
print("Numero di valori mancanti nella colonna 'Content':", missing_subcategories)

# Calcolo del numero di righe nel dataset
num_rows = len(dataset)
print("Numero di righe nel dataset:", num_rows)

# Calcolo della lunghezza ottimale della sequenza
lunghezza_media = dataset['Content'].apply(lambda x: len(x.split())).mean()
lunghezza_massima = dataset['Content'].apply(lambda x: len(x.split())).max()
lunghezza_minima = dataset['Content'].apply(lambda x: len(x.split())).min()

print("Lunghezza media della sequenza:", lunghezza_media)
print("Lunghezza massima della sequenza:", lunghezza_massima)
print("Lunghezza minima della sequenza:", lunghezza_minima)

# Conta le occorrenze di ogni categoria e sottocategoria
categoria_conta = dataset['Category'].value_counts()
sottocategoria_conta = dataset['Subcategory'].value_counts()

print("Conteggio delle Categorie:\n", categoria_conta)
print("\nConteggio delle Sottocategorie:\n", sottocategoria_conta)



# Addestramento del modello

In [ ]:
import os
from google.colab import drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/dilbert-topic-extractor')
from pandas import read_csv
from custom_layers import PositionalEncoding, TransformerEncoderLayer, F1Score
import datetime
from numpy import std as npStd, mean as npMean, unique as npUnique, argmax as npArgmax
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from tensorflow import int32
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, GlobalAveragePooling1D
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.math import argmax
from tensorflow.keras.losses import CategoricalCrossentropy
import pickle

# Carico il dataset se non è già stato caricato nella fase precedente
if 'dataset' not in locals():

    # Percorso del dataset
    dataset_path = '/content/drive/MyDrive/dilbert-topic-extractor/labeled_dataset.csv'

    # Carica il dataset
    dataset = read_csv(dataset_path)

# Definizione dei parametri
max_sequence_length = 125
num_categories = 6
num_subcategories = 18
num_heads = 4
d_model = 64
dff = 256
batch_size = 16
epochs = 40
vocab_size = 3000

# Dividi il dataset in set di addestramento e test
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42, stratify=dataset[['Category', 'Subcategory']])

# Salva il set di test in un file CSV per una successiva valutazione
test_data_path = '/content/drive/MyDrive/dilbert-topic-extractor/test_data.csv'
test_data.to_csv(test_data_path, index=False)

# Estrai testi, categorie e sottocategorie dai dati
train_texts = train_data['Content']
train_categories = train_data['Category']
train_subcategories = train_data['Subcategory']

# Tokenizzazione e padding dei testi
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts)
train_padded = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

# Salvataggio del tokenizer
tokenizer_path = '/content/drive/MyDrive/dilbert-topic-extractor/tokenizer.pickle'
with open(tokenizer_path, 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Codifica le etichette delle categorie e delle sottocategorie in interi per i dati di addestramento
category_label_encoder = LabelEncoder()
subcategory_label_encoder = LabelEncoder()

train_categories_encoded = category_label_encoder.fit_transform(train_categories)
train_subcategories_encoded = subcategory_label_encoder.fit_transform(train_subcategories)

# Salvataggio dei LabelEncoder
category_encoder_path = '/content/drive/MyDrive/dilbert-topic-extractor/category_encoder.pickle'
subcategory_encoder_path = '/content/drive/MyDrive/dilbert-topic-extractor/subcategory_encoder.pickle'
with open(category_encoder_path, 'wb') as handle:
    pickle.dump(category_label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(subcategory_encoder_path, 'wb') as handle:
    pickle.dump(subcategory_label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Calcola i pesi delle classi per le categorie
class_weights_category = compute_class_weight(
    class_weight='balanced',
    classes=npUnique(train_categories),
    y=train_categories)

class_weights_category = dict(enumerate(class_weights_category))

# Calcola i pesi anche per le sottocategorie
class_weights_subcategory = compute_class_weight(
    class_weight='balanced',
    classes=npUnique(train_subcategories),
    y=train_subcategories)

class_weights_subcategory = dict(enumerate(class_weights_subcategory))

# One-hot encoding per i dati di addestramento
train_categories_encoded = to_categorical(train_categories_encoded, num_classes=num_categories)
train_subcategories_encoded = to_categorical(train_subcategories_encoded, num_classes=num_subcategories)

# Aggiunta della regularizzazione L1/L2 per ridurre l'overfitting

regularizer = l1_l2(l1=1e-4, l2=1e-3)

# Definisci gli input del modello
text_input = Input(shape=(max_sequence_length,), dtype=int32, name='text_input')

# Embedding layer per il testo
embedding_layer = Embedding(vocab_size, d_model)(text_input)

# Aggiungi un layer LSTM
lstm_layer = LSTM(d_model, return_sequences=True, dropout=0.4)(embedding_layer)
dense_layer = Dense(dff, activation='relu', kernel_regularizer=l1_l2(l1=1e-3, l2=1e-2))

# Applica la codifica posizionale
pos_encoding_layer = PositionalEncoding(max_sequence_length, d_model)
pos_encoded = pos_encoding_layer(lstm_layer)

# Applica i tuoi strati Transformer personalizzati
transformer_output1 = TransformerEncoderLayer(num_heads, d_model, dff, rate=0.3, regularizer=regularizer)(pos_encoded)
transformer_output2 = TransformerEncoderLayer(num_heads, d_model, dff, rate=0.3, regularizer=regularizer)(transformer_output1)

# Aggiungi un layer di dropout dopo il Transformer
dropout_layer = Dropout(0.3)(transformer_output2)

# Pooled output
pooled_output = GlobalAveragePooling1D()(dropout_layer)

# Output per categorie e sottocategorie
category_output = Dense(num_categories, activation='softmax', name='category_output')(pooled_output)
subcategory_output = Dense(num_subcategories, activation='softmax', name='subcategory_output')(pooled_output)

# Creazione del modello
model = Model(inputs=text_input, outputs=[category_output, subcategory_output])

# Utilizzo dell'early stopping per evitare l'overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Aggiungi la callback per il Learning Rate Scheduler
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0001)

# Preparazione dei dati per la cross-validation
X = train_padded
Y = {'category_output': train_categories_encoded, 'subcategory_output': train_subcategories_encoded}

# Configura la cross-validation
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Struttura dati per memorizzare le metriche di ogni fold
fold_metrics = []

# Loop per la cross-validation
for fold, (train_index, test_index) in enumerate(skf.split(X, argmax(Y['category_output'], axis=1))):
    print(f"Training on fold {fold+1}/{n_splits}...")

    # Dividi i dati in training e validation per questa fold
    X_train, X_val = X[train_index], X[test_index]
    Y_train = {key: value[train_index] for key, value in Y.items()}
    Y_val = {key: value[test_index] for key, value in Y.items()}

    # Crea una directory per i log di TensorBoard per questa fold
    log_dir = f"logs/fit/folds/{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}_fold_{fold+1}"
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    # Configura il callback di TensorBoard per questa fold
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Compila il modello
    model.compile(optimizer='adam',
                  loss={'category_output': 'categorical_crossentropy', 'subcategory_output': 'categorical_crossentropy'},
                  metrics={'category_output': ['accuracy', Precision(), Recall(), F1Score(num_categories)],
                          'subcategory_output': ['accuracy', Precision(), Recall(), F1Score(num_subcategories)]},
                  loss_weights={'category_output': class_weights_category, 'subcategory_output': class_weights_subcategory})

    # Addestra il modello su questa fold
    history = model.fit(
        X_train, Y_train,
        validation_data=(X_val, Y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stopping, tensorboard_callback, lr_scheduler]
    )

    # Valutazione del modello su questo fold
    scores = model.evaluate(X_val, Y_val, verbose=0)

    # Memorizza le metriche di questo fold
    fold_metrics.append({
        'fold': fold + 1,
        'total_loss': scores[0],
        'category_output_loss': scores[1],
        'subcategory_output_loss': scores[2],
        'category_accuracy': scores[3],
        'category_precision': scores[4],
        'category_recall': scores[5],
        'category_f1_score': scores[6],
        'subcategory_accuracy': scores[7],
        'subcategory_precision': scores[8],
        'subcategory_recall': scores[9],
        'subcategory_f1_score': scores[10],
    })

    # Salvataggio del modello per questo fold
    model.save(f"model_fold_{fold+1}.keras")

# Stampa delle metriche per ogni fold
for metric in fold_metrics:
    print(f"Fold {metric['fold']}: Loss = {metric['total_loss']}, "
          f"Category Acc = {metric['category_accuracy']}, "
          f"Category Precision = {metric['category_precision']}, "
          f"Category Recall = {metric['category_recall']}, "
          f"Category F1 = {metric['category_f1_score']}, "
          f"Subcategory Acc = {metric['subcategory_accuracy']}, "
          f"Subcategory Precision = {metric['subcategory_precision']}, "
          f"Subcategory Recall = {metric['subcategory_recall']}, "
          f"Subcategory F1 = {metric['subcategory_f1_score']}")

# Calcola e stampa la media e la deviazione standard per ogni metrica
average_metrics = {key: npMean([fold[key] for fold in fold_metrics]) for key in fold_metrics[0].keys()}
std_metrics = {key: npStd([fold[key] for fold in fold_metrics]) for key in fold_metrics[0].keys()}

print("\nMedia delle metriche:", average_metrics)
print("\nDeviazione standard delle metriche:", std_metrics)

# Addestramento del modello finale su tutti i dati disponibili
model.compile(optimizer='adam',
              loss={'category_output': 'categorical_crossentropy', 'subcategory_output': 'categorical_crossentropy'},
              metrics={'category_output': ['accuracy', Precision(), Recall(), F1Score(num_categories)],
                       'subcategory_output': ['accuracy', Precision(), Recall(), F1Score(num_subcategories)]},
                  loss_weights={'category_output': class_weights_category, 'subcategory_output': class_weights_subcategory})

model.fit(X, Y, epochs=epochs, batch_size=batch_size)

# Percorso per salvare il modello finale
final_model_path = '/content/drive/MyDrive/dilbert-topic-extractor/model_final.keras'

# Salvataggio del modello finale
model.save(final_model_path)

print(f"Modello finale salvato in: {final_model_path}")

# Valutazione del modello finale
final_scores = model.evaluate(X, Y, verbose=0)

# Stampa delle metriche del modello finale
print("\nMetriche del modello finale sui dati di addestramento:")
print(f"\nLoss totale = {final_scores[0]}")
print(f"\nLoss per le categorie = {final_scores[1]}")
print(f"\nLoss per le sottocategorie = {final_scores[2]}")
print(f"\nAccuracy per le categorie = {final_scores[3]}")
print(f"\nPrecision per le categorie = {final_scores[4]}")
print(f"\nRecall per le categorie = {final_scores[5]}")
print(f"\nF1 Score per le categorie = {final_scores[6]}")
print(f"\nAccuracy per le sottocategorie = {final_scores[7]}")
print(f"\nPrecision per le sottocategorie = {final_scores[8]}")
print(f"\nRecall per le sottocategorie = {final_scores[9]}")
print(f"\nF1 Score per le sottocategorie = {final_scores[10]}")

# Avvia TensorBoard all'interno del notebook
# %load_ext tensorboard
# %tensorboard --logdir logs/fit

# Contestualizzazione automatica:

Valutazione del modello:

In [ ]:
import os
import sys
sys.path.append('/content/drive/MyDrive/dilbert-topic-extractor')
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report
from custom_layers import PositionalEncoding, TransformerEncoderLayer, F1Score
from numpy import argmax
import pickle


# Definisci il dizionario custom_objects
custom_objects = {
    'PositionalEncoding': PositionalEncoding,
    'TransformerEncoderLayer': TransformerEncoderLayer,
    'F1Score': F1Score
}

# Percorso per il modello salvato, il dataset di test, il tokenizer e gli encoders
model_path = '/content/drive/MyDrive/dilbert-topic-extractor/model_final.keras'
test_data_path = '/content/drive/MyDrive/dilbert-topic-extractor/test_data.csv'
tokenizer_path = '/content/drive/MyDrive/dilbert-topic-extractor/tokenizer.pickle'
category_encoder_path = '/content/drive/MyDrive/dilbert-topic-extractor/category_encoder.pickle'
subcategory_encoder_path = '/content/drive/MyDrive/dilbert-topic-extractor/subcategory_encoder.pickle'

# Carica il modello
model = load_model('/content/drive/MyDrive/dilbert-topic-extractor/model_final.keras', custom_objects=custom_objects)

# Carica il tokenizer e gli encoder salvati
with open(tokenizer_path, 'rb') as handle:
    tokenizer = pickle.load(handle)
with open(category_encoder_path, 'rb') as handle:
    category_label_encoder = pickle.load(handle)
with open(subcategory_encoder_path, 'rb') as handle:
    subcategory_label_encoder = pickle.load(handle)

# Carica i dati di test
test_data = pd.read_csv(test_data_path)
test_texts = test_data['Content']
test_categories = test_data['Category']
test_subcategories = test_data['Subcategory']

# Tokenizzazione e padding dei testi di test
test_sequences = tokenizer.texts_to_sequences(test_texts)
test_padded = pad_sequences(test_sequences, maxlen=125, padding='post', truncating='post')

# Codifica le etichette per i dati di test usando gli encoder salvati
test_categories_encoded = category_label_encoder.transform(test_categories)
test_subcategories_encoded = subcategory_label_encoder.transform(test_subcategories)

# One-hot encoding per i dati di test
test_categories_encoded = to_categorical(test_categories_encoded, num_classes=6)
test_subcategories_encoded = to_categorical(test_subcategories_encoded, num_classes=18)

# Valutazione del modello sui dati di test
evaluation_results = model.evaluate(test_padded, [test_categories_encoded, test_subcategories_encoded])

# Stampa delle metriche
print("Metriche del modello finale sui dati di test:")
print(f"\nLoss totale = {evaluation_results[0]}")
print(f"\nLoss per le categorie = {evaluation_results[1]}")
print(f"\nLoss per le sottocategorie = {evaluation_results[2]}")
print(f"\nAccuracy per le categorie = {evaluation_results[3]}")
print(f"\nPrecision per le categorie = {evaluation_results[4]}")
print(f"\nRecall per le categorie = {evaluation_results[5]}")
print(f"\nF1 Score per le categorie = {evaluation_results[6]}")
print(f"\nAccuracy per le sottocategorie = {evaluation_results[7]}")
print(f"\nPrecision per le sottocategorie = {evaluation_results[8]}")
print(f"\nRecall per le sottocategorie = {evaluation_results[9]}")
print(f"\nF1 Score per le sottocategorie = {evaluation_results[10]}")

# Effettua le previsioni
predictions = model.predict(test_padded)

# Utilizza le etichette originali in formato numerico per calcolare la matrice di confusione e il classification report
predicted_categories = argmax(predictions[0], axis=1)
predicted_subcategories = argmax(predictions[1], axis=1)

# Calcola la matrice di confusione e il classification report
cm_categories = confusion_matrix(category_label_encoder.transform(test_categories), predicted_categories)
report_categories = classification_report(category_label_encoder.transform(test_categories), predicted_categories, target_names=category_label_encoder.classes_, zero_division=1)

cm_subcategories = confusion_matrix(subcategory_label_encoder.transform(test_subcategories), predicted_subcategories)
report_subcategories = classification_report(subcategory_label_encoder.transform(test_subcategories), predicted_subcategories, target_names=subcategory_label_encoder.classes_, zero_division=1)

# Stampa la matrice di confusione e il classification report
print("\nMatrice di Confusione per le Categorie:")
print(cm_categories)
print("\nClassification Report per le Categorie:")
print(report_categories)

print("\nMatrice di Confusione per le Sottocategorie:")
print(cm_subcategories)
print("\nClassification Report per le Sottocategorie:")
print(report_subcategories)

Predizione su singolo file:

In [ ]:
import os
import sys
sys.path.append('/content/drive/MyDrive/dilbert-topic-extractor')
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from custom_layers import PositionalEncoding, TransformerEncoderLayer, F1Score
from numpy import argmax
import pickle

# Definisci il dizionario custom_objects
custom_objects = {
    'PositionalEncoding': PositionalEncoding,
    'TransformerEncoderLayer': TransformerEncoderLayer,
    'F1Score': F1Score
}

# Percorso per il modello salvato, il dataset di test, il tokenizer e gli encoders
model_path = '/content/drive/MyDrive/dilbert-topic-extractor/model_final.keras'
test_data_path = '/content/drive/MyDrive/dilbert-topic-extractor/test_data.csv'
tokenizer_path = '/content/drive/MyDrive/dilbert-topic-extractor/tokenizer.pickle'
category_encoder_path = '/content/drive/MyDrive/dilbert-topic-extractor/category_encoder.pickle'
subcategory_encoder_path = '/content/drive/MyDrive/dilbert-topic-extractor/subcategory_encoder.pickle'

# Carica il modello
model = load_model('/content/drive/MyDrive/dilbert-topic-extractor/model_final.keras', custom_objects=custom_objects)

# Carica il tokenizer e gli encoder salvati
with open(tokenizer_path, 'rb') as handle:
    tokenizer = pickle.load(handle)
with open(category_encoder_path, 'rb') as handle:
    category_label_encoder = pickle.load(handle)
with open(subcategory_encoder_path, 'rb') as handle:
    subcategory_label_encoder = pickle.load(handle)

# Leggi il testo dal file
file_path = '/content/drive/MyDrive/dilbert-topic-extractor/test-files/1993-01-16.txt'
with open(file_path, 'r') as file:
    new_text = file.read()

# Preparazione del nuovo testo per la predizione
sequences = tokenizer.texts_to_sequences([new_text])
padded = pad_sequences(sequences, maxlen=125, padding='post', truncating='post')

# Eseguo la predizione
predictions = model.predict(padded)

# Decodifico le predizioni
predicted_categories = category_label_encoder.inverse_transform(argmax(predictions[0], axis=-1))
predicted_subcategories = subcategory_label_encoder.inverse_transform(argmax(predictions[1], axis=-1))

print("Categoria Predetta:", predicted_categories[0])
print("Sottocategoria Predetta:", predicted_subcategories[0])


Predizione su file multipli e generazione di un nuovo dataset con le predizioni: (Analisi di stabilità)

In [ ]:
import os
import sys
sys.path.append('/content/drive/MyDrive/dilbert-topic-extractor')
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from custom_layers import PositionalEncoding, TransformerEncoderLayer, F1Score
from numpy import argmax
import pickle
from datetime import datetime

# Definisci il dizionario custom_objects
custom_objects = {
    'PositionalEncoding': PositionalEncoding,
    'TransformerEncoderLayer': TransformerEncoderLayer,
    'F1Score': F1Score
}

# Percorso per il modello salvato, il dataset di test, il tokenizer e gli encoders
model_path = '/content/drive/MyDrive/dilbert-topic-extractor/model_final.keras'
test_data_path = '/content/drive/MyDrive/dilbert-topic-extractor/test_data.csv'
tokenizer_path = '/content/drive/MyDrive/dilbert-topic-extractor/tokenizer.pickle'
category_encoder_path = '/content/drive/MyDrive/dilbert-topic-extractor/category_encoder.pickle'
subcategory_encoder_path = '/content/drive/MyDrive/dilbert-topic-extractor/subcategory_encoder.pickle'

# Carica il modello salvato
model = load_model('/content/drive/MyDrive/dilbert-topic-extractor/model_final.keras', custom_objects=custom_objects)

# Carica il tokenizer e gli encoder salvati
with open(tokenizer_path, 'rb') as handle:
    tokenizer = pickle.load(handle)
with open(category_encoder_path, 'rb') as handle:
    category_label_encoder = pickle.load(handle)
with open(subcategory_encoder_path, 'rb') as handle:
    subcategory_label_encoder = pickle.load(handle)

# Genera un timestamp
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")

# Percorso della directory con i file
directory_path = '/content/drive/MyDrive/dilbert-topic-extractor/test-files'

# Preparazione del dataset
dataset = []

# Elenca e processa ogni file nella directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path):

        # Leggi il contenuto del file
        with open(file_path, 'r') as file:
            content = file.read()

        # Preparazione del testo per la predizione
        sequences = tokenizer.texts_to_sequences([content])
        padded = pad_sequences(sequences, maxlen=125, padding='post', truncating='post')

        # Esegui la predizione
        predictions = model.predict(padded)

        # Decodifica le predizioni
        predicted_category = category_label_encoder.inverse_transform(argmax(predictions[0], axis=-1))[0]
        predicted_subcategory = subcategory_label_encoder.inverse_transform(argmax(predictions[1], axis=-1))[0]

        # Aggiungi al dataset
        dataset.append({
            "File Name": filename,
            "Predicted Category": predicted_category,
            "Predicted Subcategory": predicted_subcategory,
            "Content": content
        })

# Converti il dataset in un DataFrame di pandas
df = pd.DataFrame(dataset)

# Salva il DataFrame in un file CSV
csv_file_path = f'/content/drive/MyDrive/dilbert-topic-extractor/predicted_dataset_{timestamp}.csv'
df.to_csv(csv_file_path, index=False)